# Sentiment Classification

The task is from [aidea](https://aidea-web.tw/topic/c4a666bb-7d83-45a6-8c3b-57514faf2901), the goal is to predict the sentiment of each article.

In [1]:
import torch
import numpy as np
import pandas as pd

## Data Preparation

In [2]:
from collections import Counter
from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer('basic_english')

In [3]:
imdb_df = pd.read_csv('../data/imdb.csv')
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [4]:
imdb_df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [6]:
df = pd.merge(imdb_df, test_df, on='review', how='left')

In [9]:
test_df.count()

ID        29341
review    29341
dtype: int64

In [11]:
df['ID'].count() / len(test_df)

0.8675914249684742

In [96]:
train_df

,ID,review,sentiment
0,41411,I watched this film because I'm a big fan of R...,0
1,37586,It does not seem that this movie managed to pl...,1
2,6017,"Enough is not a bad movie , just mediocre .",0
3,44656,my friend and i rented this one a few nights a...,0
4,38711,"Just about everything in this movie is wrong, ...",0
...,...,...,...
29336,8019,It 's one of the most honest films ever made a...,1
29337,453,An absorbing and unsettling psychological drama .,1
29338,13097,"Soylent Green IS...a really good movie, actual...",1
29339,26896,There just isn't enough here. There a few funn...,0


In [97]:
train_df.head()

,ID,review,sentiment
0,41411,I watched this film because I'm a big fan of R...,0
1,37586,It does not seem that this movie managed to pl...,1
2,6017,"Enough is not a bad movie , just mediocre .",0
3,44656,my friend and i rented this one a few nights a...,0
4,38711,"Just about everything in this movie is wrong, ...",0


In [107]:
imdb_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [106]:
imdb_df['sentiment'] = imdb_df['sentiment'].replace(['negative', 'positive'], [0, 1])

In [113]:
df = pd.concat([train_df[['review', 'sentiment']], imdb_df], axis=0)

In [114]:
df['review'].value_counts()

Hilarious, clean, light-hearted, and quote-worthy. What else can you ask for in a film? This is my all-time, number one favorite movie. Ever since I was a little girl, I've dreamed of owning a blue van with flames and an observation bubble.<br /><br />The cliché characters in ridiculous situations are what make this film such great fun. The wonderful comedic chemistry between Stephen Furst (Harold) and Andy Tennant (Melio) make up most of my favorite parts of the movie. And who didn't love the hopeless awkwardness of Flynch? Don't forget the airport antics of Leon's cronies, dressed up as Hari Krishnas: dancing, chanting and playing the tambourine--unbeatable! The clues are genius, the locations are classic, and the plot is timeless.<br /><br />A word to the wise, if you didn't watch this film when you were little, it probably won't win a place in your heart today. But nevertheless give it a chance, you may find that "It doesn't matter what you say, it doesn't matter what you do, you'v

In [175]:
drop_df.drop_duplicates(subset=('clean_text')).reindex(columns=('review', 'clean_text', 'sentiment')).to_csv('../data/all.csv', index=False)

In [115]:
drop_df = df.drop_duplicates(subset=('review'))

In [170]:
drop_df.to_csv('../data/all.csv', index=False)

In [ ]:
drop_df.reindex()

In [117]:
drop_df.head()

,review,sentiment
0,I watched this film because I'm a big fan of R...,0
1,It does not seem that this movie managed to pl...,1
2,"Enough is not a bad movie , just mediocre .",0
3,my friend and i rented this one a few nights a...,0
4,"Just about everything in this movie is wrong, ...",0


In [118]:
drop_df['sentiment'].value_counts()

1    27354
0    26606
Name: sentiment, dtype: int64

In [36]:
train_df['len'] = 0

In [37]:
lens = list()

for text in train_df['review']:
    lens += [len(tokenizer(text))]

In [38]:
train_df['len'] = lens

In [49]:
train_df[train_df['len'] > 600]['sentiment'].value_counts()

1    957
0    830
Name: sentiment, dtype: int64

In [8]:
train_df.describe()

,ID,sentiment
count,29341.000000,29341.000000
mean,29348.411097,0.509662
std,17002.074346,0.499915
min,4.000000,0.000000
25%,14564.000000,0.000000
50%,29348.000000,1.000000
75%,44162.000000,1.000000
max,58681.000000,1.000000


In [9]:
train_df['sentiment'].value_counts()

1    14954
0    14387
Name: sentiment, dtype: int64

In [7]:
test_df

,ID,review
0,22622,Robert Lansing plays a scientist experimenting...
1,10162,"Well I've enjoy this movie, even though someti..."
2,17468,First things first - though I believe Joel Sch...
3,42579,I watched this movie on the grounds that Amber...
4,701,A certain sexiness underlines even the dullest...
...,...,...
29336,30370,It is difficult to rate a writer/director's fi...
29337,18654,"After watching this movie once, it quickly bec..."
29338,47985,"Even though i sat and watched the whole thing,..."
29339,9866,Warning Spoilers following. Superb recreation ...


In [10]:
sample_df = pd.read_csv('../data/sample_submission.csv')

In [11]:
sample_df

,ID,sentiment
0,22622,1
1,10162,1
2,17468,1
3,42579,1
4,701,1
...,...,...
29336,30370,1
29337,18654,1
29338,47985,1
29339,9866,1


### Data Split

In [65]:
from sklearn.model_selection import KFold # import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=123) # Define the split - into 2 folds 
kf.get_n_splits(train_df) # returns the number of splitting iterations in the cross-validator

10

In [41]:
X.iloc[[0,1,2]]

,ID,review,sentiment
0,41411,I watched this film because I'm a big fan of R...,0
1,37586,It does not seem that this movie managed to pl...,1
2,6017,"Enough is not a bad movie , just mediocre .",0


In [45]:
X, y = train_df, train_df['sentiment'].to_numpy()
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [0 1 2 3 5 6 7 8 9] TEST: [4]
9 1
TRAIN: [1 2 3 4 5 6 7 8 9] TEST: [0]
9 1
TRAIN: [0 1 2 3 4 5 6 8 9] TEST: [7]
9 1
TRAIN: [0 1 2 3 4 6 7 8 9] TEST: [5]
9 1
TRAIN: [0 1 2 3 4 5 6 7 9] TEST: [8]
9 1
TRAIN: [0 1 2 4 5 6 7 8 9] TEST: [3]
9 1
TRAIN: [0 2 3 4 5 6 7 8 9] TEST: [1]
9 1
TRAIN: [0 1 2 3 4 5 7 8 9] TEST: [6]
9 1
TRAIN: [0 1 2 3 4 5 6 7 8] TEST: [9]
9 1
TRAIN: [0 1 3 4 5 6 7 8 9] TEST: [2]
9 1


### Preprocessing

In [123]:
import re
import string
# Removing all punctuations from Text
mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

def word_replace(text):
    return text.replace('<br />','')

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)


def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def preprocess(text):
    text=clean_contractions(text,mapping)
    text=text.lower()
    text=word_replace(text)
    text=remove_urls(text)
    text=remove_html(text)
#     text=remove_stopwords(text)
    text=remove_punctuation(text)
#     text=stem_words(text) ## Takes too much of time
#     text=lemmatize_words(text)
    return text

In [124]:
drop_df['clean_text'] = drop_df['review'].apply(lambda x: preprocess(x))

/var/folders/fp/kr0zqp696h3bm6dmv7lmb_jc0000gn/T/ipykernel_8573/3308018844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_df['clean_text'] = drop_df['review'].apply(lambda x: preprocess(x))


In [128]:
train_df['clean_text'] = train_df['review'].apply(lambda x: preprocess(x))

In [155]:
test_df['clean_text'] = test_df['review'].apply(lambda x: preprocess(x))

In [127]:
drop_df['clean_text'][:10]

0    i watched this film because i am a big fan of ...
1    it does not seem that this movie managed to pl...
2            enough is not a bad movie  just mediocre 
3    my friend and i rented this one a few nights a...
4    just about everything in this movie is wrong w...
5    this is not bonnie and clyde or thelma and lou...
6    i have to say that i really liked under siege ...
7    kramer vs kramer is a nearheartening drama abo...
8    like the other comments says this might be sur...
9    the tunes are the best aspect of this televisi...
Name: clean_text, dtype: object

In [119]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

tokenizer = get_tokenizer('basic_english')
counter = Counter()
for line in train_df['review']:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

In [121]:
Vocab(Counter(dict(counter.most_common(10))))

In [49]:
len(counter)

102969

In [55]:
[vocab[i] for i in ['I', 'am', 'aaaaaaaaaaaaa', '<pad>']]

[0, 241, 0, 1]

In [101]:
def text_pipeline(X):
    if isinstance(X, list):
        return [[vocab[i] for i in tokenizer(preprocess(text))] for text in X]
    return [vocab[i] for i in tokenizer(X)]

In [102]:
text_pipeline("I am a good boy! ADJISDAKSD unkqwjs <pad> <pad>")

NameError: name 'vocab' is not defined

In [67]:
text_pipeline(["I am a good", "boy!"])

[[13, 241, 5, 57], [412, 36]]

### Data Iteration

In [255]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch, use_bag=False):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         label_list.append(_label)
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    if use_bag:
        offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
        text_list = torch.cat(text_list)
    else:
        offsets = torch.tensor(offsets[1:], dtype=torch.int64)
        text_list = pad_sequence(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

df = train_df.iloc[:100]
train_iter = list(zip(df['review'], df['sentiment']))
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [154]:
for i in dataloader:
    print(i[0].shape, i[1].shape, i[2].shape)
    print(i)

torch.Size([8]) torch.Size([513, 8]) torch.Size([8])
torch.Size([8]) torch.Size([513, 8]) torch.Size([8])
(tensor([0, 1, 0, 0, 0, 1, 0, 1]), tensor([[  13,   11,  197,  ...,   14,   13, 5130],
        [ 295,  130,   10,  ...,  213,   33, 1483],
        [  14,   29,   29,  ...,    9,    8,    3],
        ...,
        [   0,  272,    0,  ...,    0,    0,    0],
        [   0,  319,    0,  ...,    0,    0,    0],
        [   0,    3,    0,  ...,    0,    0,    0]]), tensor([210, 513,  10, 233, 237, 276,  94, 309]))
torch.Size([8]) torch.Size([537, 8]) torch.Size([8])
torch.Size([8]) torch.Size([537, 8]) torch.Size([8])
(tensor([0, 1, 0, 1, 1, 1, 0, 0]), tensor([[   45,     2,    14,  ...,    51,  4700,    18],
        [    2,  3725,   373,  ...,    26,    12,  3034],
        [   87,    31,   119,  ...,   531,   732,    18],
        ...,
        [    0,     0, 85389,  ...,     0,     0,     0],
        [    0,     0,   373,  ...,     0,     0,     0],
        [    0,     0,     3,  ...,   

## Model Selection

In order to validate the performance of model, the frequently adopted solutions are cross validation and the usage of validation set. Since the size of training samples is small, cross validation would be a more appropriate strategy.

1. RNN-based model
2. Naive-bayes model
3. Bert model

In [134]:
from sklearn.model_selection import StratifiedKFold

In [139]:
skf = StratifiedKFold(n_splits=10)

### Baseline -TFiDF

In [51]:
train_df['review']

0        I watched this film because I'm a big fan of R...
1        It does not seem that this movie managed to pl...
2              Enough is not a bad movie , just mediocre .
3        my friend and i rented this one a few nights a...
4        Just about everything in this movie is wrong, ...
                               ...                        
29336    It 's one of the most honest films ever made a...
29337    An absorbing and unsettling psychological drama .
29338    Soylent Green IS...a really good movie, actual...
29339    There just isn't enough here. There a few funn...
29340    This show was absolutely terrible. For one Geo...
Name: review, Length: 29341, dtype: object

In [129]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', token_pattern=r'(?u)\b[A-Za-z]+\b')
vectorizer.fit(drop_df['clean_text'])

TfidfVectorizer(stop_words='english', token_pattern='(?u)\\b[A-Za-z]+\\b')

In [131]:
dir(vectorizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_char_ngrams',
 '_char_wb_ngrams',
 '_check_n_features',
 '_check_params',
 '_check_stop_words_consistency',
 '_check_vocabulary',
 '_count_vocab',
 '_get_param_names',
 '_get_tags',
 '_limit_features',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_sort_features',
 '_stop_words_id',
 '_tfidf',
 '_validate_data',
 '_validate_params',
 '_validate_vocabulary',
 '_warn_for_unused_params',
 '_white_spaces',
 '_word_ngrams',
 'analyzer',
 'binary',
 'build_analyzer',
 'build_preprocessor',
 'build_tokenizer',
 'decode',
 'decode_error',
 'dtype',
 'encoding',
 'f

In [73]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english', token_pattern=r'(?u)\b[A-Za-z]+\b')
cv.fit(train_df['review'])

CountVectorizer(stop_words='english', token_pattern='(?u)\\b[A-Za-z]+\\b')

In [133]:
len(vectorizer.get_feature_names())

212617

In [137]:
X, y = drop_df['clean_text'], drop_df['sentiment']

In [144]:
X[[1,2,3]]

1    it does not seem that this movie managed to pl...
1    a wonderful little production the filming tech...
2            enough is not a bad movie  just mediocre 
3    my friend and i rented this one a few nights a...
Name: clean_text, dtype: object

In [68]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression

In [154]:
def train(model, vectorizer, X, y):
    prf = {
        'accuracy': list(),
        'precision': list(),
        'recall': list(),
        'f1-score': list()
    }

    for idx, (train_index, valid_index) in enumerate(skf.split(X, y)):
        print(f"Cross validation {idx}-fold")
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_valid, y_valid = X.iloc[valid_index], y.iloc[valid_index]

        clf = model.fit(vectorizer.transform(X_train), y_train)
        X_valid_transform = vectorizer.transform(X_valid)
        y_preds = clf.predict(X_valid_transform)
        results = precision_recall_fscore_support(y_valid, y_preds, average='binary')
        print(results)
        prf['accuracy'] += [clf.score(X_valid_transform, y_valid)]
        prf['precision'] += [results[0]]
        prf['recall'] += [results[1]]
        prf['f1-score'] += [results[2]]
    return prf

#### Tfidf + Logistic Regression

In [141]:
from sklearn.linear_model import LogisticRegression

In [148]:
model = LogisticRegression()
prf = train(model, vectorizer, X, y)

[ 5329  5334  5337 ... 53957 53958 53959] [   0    1    2 ... 5452 5453 5458]
Cross validation 0-fold
(0.8533287101248266, 0.89981718464351, 0.8759565759031855, None)
[    0     1     2 ... 53957 53958 53959] [ 5329  5334  5337 ... 10858 10859 10860]
Cross validation 1-fold
(0.8612273361227336, 0.903107861060329, 0.8816705336426913, None)
[    0     1     2 ... 53957 53958 53959] [10728 10729 10730 ... 16310 16311 16312]
Cross validation 2-fold
(0.8589518114667605, 0.8928702010968922, 0.8755826461097168, None)
[    0     1     2 ... 53957 53958 53959] [16059 16060 16063 ... 21797 21800 21802]
Cross validation 3-fold
(0.8712800286841161, 0.8884826325411335, 0.8797972483707459, None)
[    0     1     2 ... 53957 53958 53959] [21366 21367 21372 ... 27154 27158 27160]
Cross validation 4-fold
(0.8610229276895943, 0.8925045703839123, 0.8764811490125674, None)
[    0     1     2 ... 53957 53958 53959] [26763 26770 26771 ... 32541 32546 32547]
Cross validation 5-fold
(0.8765652951699463, 0.895

In [149]:
for k, v in prf.items():
    print(k, np.mean(v))

accuracy 0.8825796886582655
precision 0.8723164699160139
recall 0.900306015266686
f1-score 0.8860549514855641


In [156]:
model = LogisticRegression()

In [157]:
model.fit(vectorizer.transform(X), y)

LogisticRegression()

In [159]:
preds = model.predict(vectorizer.transform(test_df['clean_text']))

In [160]:
preds

array([0, 1, 0, ..., 0, 0, 0])

In [161]:
model.score(vectorizer.transform(X), y)

0.9273350630096368

In [162]:
submit_df = test_df
submit_df['sentiment'] = preds

In [163]:
submit_df.head()

,ID,review,clean_text,sentiment
0,22622,Robert Lansing plays a scientist experimenting...,robert lansing plays a scientist experimenting...,0
1,10162,"Well I've enjoy this movie, even though someti...",well i have enjoy this movie even though somet...,1
2,17468,First things first - though I believe Joel Sch...,first things first though i believe joel schu...,0
3,42579,I watched this movie on the grounds that Amber...,i watched this movie on the grounds that amber...,0
4,701,A certain sexiness underlines even the dullest...,a certain sexiness underlines even the dullest...,1


In [165]:
submit_df = submit_df.drop(columns=['review', 'clean_text'])

In [166]:
submit_df.to_csv('submit.csv', index=False)

#### Tfidf + RandomForest

In [75]:
from sklearn.ensemble import RandomForestClassifier

In [152]:
model = RandomForestClassifier(n_jobs=-1)
prf = train(model, vectorizer, X, y)

[ 5329  5334  5337 ... 53957 53958 53959] [   0    1    2 ... 5452 5453 5458]
Cross validation 0-fold
(0.8253521126760563, 0.8570383912248629, 0.8408968609865471, None)
[    0     1     2 ... 53957 53958 53959] [ 5329  5334  5337 ... 10858 10859 10860]
Cross validation 1-fold
(0.8356115107913669, 0.8493601462522852, 0.842429737080689, None)
[    0     1     2 ... 53957 53958 53959] [10728 10729 10730 ... 16310 16311 16312]
Cross validation 2-fold
(0.8070475538829969, 0.8625228519195612, 0.8338635560268647, None)
[    0     1     2 ... 53957 53958 53959] [16059 16060 16063 ... 21797 21800 21802]
Cross validation 3-fold
(0.8240312833274085, 0.8475319926873858, 0.8356164383561645, None)
[    0     1     2 ... 53957 53958 53959] [21366 21367 21372 ... 27154 27158 27160]
Cross validation 4-fold
(0.8162199791159067, 0.8574040219378428, 0.8363052781740372, None)
[    0     1     2 ... 53957 53958 53959] [26763 26770 26771 ... 32541 32546 32547]
Cross validation 5-fold
(0.8534798534798534, 0.8

In [153]:
for k, v in prf.items():
    print(k, np.mean(v))

accuracy 0.8441808747220163
precision 0.8412012893432861
recall 0.8546098602691983
f1-score 0.8477041511831489


#### Tfidf + MLP

In [85]:
from sklearn.neural_network import MLPClassifier

In [88]:
model = MLPClassifier()
prf = train(model, vectorizer, X)

Cross validation 0-fold


/Users/hlu/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


(0.8485639686684073, 0.8742434431741762, 0.8612123219609142, None)
Cross validation 1-fold


KeyboardInterrupt: 

### Baseline - MLP

In [87]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [88]:
train_iter = list(zip(train_df['review'], train_df['sentiment']))
num_class = len(set([label for (text, label) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [135]:
import time
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, F.one_hot(label, num_classes=num_class).type(torch.FloatTensor))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_count = 0
    all_preds, all_labels = list(), list()
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            predicted_label = predicted_label.argmax(1)
            all_preds += [predicted_label.detach().numpy()]
            all_labels += [label.detach().numpy()]
            total_count += label.size(0)
    print(all_preds, all_labels, total_count)
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    prf = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    print(prf)
    return (all_preds == all_labels).mean()

In [136]:
from torch.utils.data.dataset import random_split
# Hyperparameters
EPOCHS = 1 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

# train_iter = list(zip(train_df.iloc[:1000]['review'], train_df.iloc[:1000]['sentiment']))

X, y = train_df.iloc[:1000]['review'], train_df['sentiment'].iloc[:1000].to_numpy()
# for train_index, test_index in kf.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y[train_index], y[test_index]

# train_dataset = list(train_iter)
# test_dataset = list(test_iter)
# num_train = int(len(train_dataset) * 0.95)
# split_train_, split_valid_ = \
#     random_split(train_dataset, [num_train, len(train_dataset) - num_train])

# train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
#                               shuffle=True, collate_fn=collate_batch)
# valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
#                               shuffle=True, collate_fn=collate_batch)
# test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
#                              shuffle=True, collate_fn=collate_batch)
print(model)
total_acc = []
for idx, (train_index, valid_index) in enumerate(kf.split(X)):
#     print("TRAIN:", train_index, "TEST:", test_index)
    print(f"Cross validation {idx}-fold")
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    train_iter = list(zip(X_train, y_train))
    valid_iter = list(zip(X_valid, y_valid))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE,
                          shuffle=True, collate_fn=collate_batch)
    valid_dataloader = DataLoader(valid_iter, batch_size=BATCH_SIZE,
                                  shuffle=True, collate_fn=collate_batch)
    cross_acc = None
    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        train(train_dataloader)
        acc = evaluate(valid_dataloader)
        if cross_acc is not None and cross_acc > acc:
            scheduler.step()
        else:
            cross_acc = acc
        print('-' * 59)
        print('| end of epoch {:3d} | time: {:5.2f}s | '
              'valid accuracy {:8.3f} '.format(epoch,
                                               time.time() - epoch_start_time,
                                                acc))
        print('-' * 59)
    total_acc += [cross_acc]
print(total_acc)
print(np.mean(total_acc))

TextClassificationModel(
  (embedding): EmbeddingBag(102971, 64, mode=mean)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)
Cross validation 0-fold
[array([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]), array([0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0])] [array([1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1]), array([0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0])] 100
(0.6933333333333334, 1.0, 0.8188976377952756, None)
-----------------------------------------------------------
| end of epoch   1 | time:  0.45s | valid ac

### 1. RNN-based model

In [256]:
import torch
from torch import nn
from torch.nn import functional as F


class Attention(nn.Module):
    # target is hidden_size
    def __init__(self, hidden_size, method='concat'):
        super(Attention, self).__init__()
        self.method = method
        if method not in ('dot', 'general', 'concat'):
            raise NotImplemented
        if method == 'general':
            self.attn = nn.Linear(hidden_size, hidden_size)
        elif method == 'concat':
            self.attn = nn.Linear(2 * hidden_size, hidden_size)
            self.v = nn.Linear(hidden_size, 1, bias=False)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        if hasattr(self, 'attn'):
            self.attn.weight.data.uniform_(-initrange, initrange)
            self.attn.bias.data.zero_()
        if hasattr(self, 'v'):
            self.v.weight.data.uniform_(-initrange, initrange)

    def dot_score(self, hidden, encoder_output):
        return torch.matmul(hidden, encoder_output)

    def general_score(self, hidden, encoder_output):
        attn = self.attn(encoder_output)
        return torch.matmul(hidden, attn)

    def concat_score(self, hidden, encoder_output):
        hidden_reshape = torch.unsqueeze(hidden, dim=0).repeat(encoder_output.size(0), 1, 1)
        attn = self.attn(torch.cat([hidden_reshape, encoder_output], dim=-1)).tanh()
        return self.v(attn).squeeze(dim=-1)

    def forward(self, hidden, encoder_output):
        # output = [lengths x batch_size x hidden_size]
        # hidden = [batch_size x hidden_size]
        attn_scores = None
        if self.method == 'dot':
            attn_scores = self.dot_score(hidden, encoder_output)
        elif self.method == 'general':
            attn_scores = self.general_score(hidden, encoder_output)
        elif self.method == 'concat':
            attn_scores = self.concat_score(hidden, encoder_output)

        # [lengths x batch_size] -> [batch_size x lengths]
        attn_scores = attn_scores.t()
        # return [batch_size x 1 x lengths]
        return F.softmax(attn_scores, dim=-1).unsqueeze(1)


In [257]:
t = torch.tensor([5,3,7,2,1])
sorted_t, idx = t.sort(descending=True)
print(sorted_t)
print(torch.gather(t, 0, torch.arange(0, idx.shape[0], dtype=torch.int64)))

tensor([7, 5, 3, 2, 1])
tensor([5, 3, 7, 2, 1])


In [261]:
from torch import nn
from torch.nn import init


def sort_sequence(inputs, lengths):
    sorted_lengths, sorted_idx = lengths.sort(descending=True)
    return inputs[sorted_idx], sorted_lengths, sorted_idx


class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, n_layers, dropout, num_classes, attention_mode, padding_idx=1):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, sparse=True, padding_idx=1)
        self.lstm = nn.LSTM(embed_size, hidden_size, n_layers, dropout=dropout, bidirectional=True, batch_first=True)
        self.attn = Attention(2 * hidden_size, attention_mode)
        self.fc = nn.Linear(2 * hidden_size, num_classes)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        for param in self.lstm.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.uniform_(param.data, -initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, text_lengths, hidden=None):
        sorted_lengths, sorted_idx = text_lengths.sort(descending=True)
        sorted_text = torch.index_select(text, -1, sorted_idx)
        emb = self.embedding(sorted_text)
        packed = nn.utils.rnn.pack_padded_sequence(emb, sorted_lengths)
        outputs, hidden = self.lstm(packed, hidden)
        hidden_state, _ = hidden
        hidden_state = hidden_state[-2:,:,:].view(1, -1, 2 * hidden_size).squeeze(0)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        attn_weights = self.attn(hidden_state, outputs)
        # attn_weights = [batch_size x 1 x lengths]
        context = torch.bmm(attn_weights, outputs.transpose(0, 1)).squeeze(1)
        output = self.fc(context)
        output = torch.index_select(output, 0, torch.arange(0, sorted_idx.shape[0], dtype=torch.int64))
        return output, hidden

In [262]:
embed_size = 50
hidden_size = 256
n_layers = 2
dropout = 0.1
num_classes = 2
attention_mode = 'concat'
model = LSTMModel(vocab_size, embed_size, hidden_size, n_layers, 
                  dropout, num_classes, attention_mode).to(device)

In [264]:
import time
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label, _ = model(text, offsets)
        loss = criterion(predited_label, F.one_hot(label, num_classes=num_class).type(torch.FloatTensor))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_count = 0
    all_preds, all_labels = list(), list()
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label, _ = model(text, offsets)
            predicted_label = predicted_label.argmax(1)
            all_preds += [predicted_label.detach().numpy()]
            all_labels += [label.detach().numpy()]
            total_count += label.size(0)
    print(all_preds, all_labels, total_count)
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    prf = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    print(prf)
    return (all_preds == all_labels).mean()

In [ ]:
from torch.utils.data.dataset import random_split
# Hyperparameters
EPOCHS = 1 # epoch
LR = 5  # learning rate
BATCH_SIZE = 32 # batch size for training

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

X, y = train_df.iloc[:1000]['review'], train_df['sentiment'].iloc[:1000].to_numpy()
print(model)
total_acc = []
for idx, (train_index, valid_index) in enumerate(kf.split(X)):
#     print("TRAIN:", train_index, "TEST:", test_index)
    print(f"Cross validation {idx}-fold")
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    train_iter = list(zip(X_train, y_train))
    valid_iter = list(zip(X_valid, y_valid))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE,
                          shuffle=True, collate_fn=collate_batch)
    valid_dataloader = DataLoader(valid_iter, batch_size=BATCH_SIZE,
                                  shuffle=True, collate_fn=collate_batch)
    cross_acc = None
    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        train(train_dataloader)
        acc = evaluate(valid_dataloader)
        if cross_acc is not None and cross_acc > acc:
            scheduler.step()
        else:
            cross_acc = acc
        print('-' * 59)
        print('| end of epoch {:3d} | time: {:5.2f}s | '
              'valid accuracy {:8.3f} '.format(epoch,
                                               time.time() - epoch_start_time,
                                                acc))
        print('-' * 59)
    total_acc += [cross_acc]
print(total_acc)
print(np.mean(total_acc))

LSTMModel(
  (embedding): Embedding(102971, 50, padding_idx=1, sparse=True)
  (lstm): LSTM(50, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (attn): Attention(
    (attn): Linear(in_features=1024, out_features=512, bias=True)
    (v): Linear(in_features=512, out_features=1, bias=False)
  )
  (fc): Linear(in_features=512, out_features=2, bias=True)
)
Cross validation 0-fold
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([4, 2])
torch.Size([32, 2])

### Bert-based Model

In [1]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [4]:
text = "I am a good person, and how about you?"
print(' Original: ', text)
print('Tokenized: ', tokenizer.tokenize(text))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(["[CLS]"] + tokenizer.tokenize(text)+ ["SEP"])

 Original:  I am a good person, and how about you?
Tokenized:  ['i', 'am', 'a', 'good', 'person', ',', 'and', 'how', 'about', 'you', '?']
Token IDs:  [101, 1045, 2572, 1037, 2204, 2711, 1010, 1998, 2129, 2055, 2017, 1029, 100]


In [5]:
input_ids = tokenizer.encode(text, add_special_tokens=True)

In [6]:
input_ids

[101, 1045, 2572, 1037, 2204, 2711, 1010, 1998, 2129, 2055, 2017, 1029, 102]

In [7]:
tokenizer.encode_plus(
                text, add_special_tokens=True, max_length=64,
                pad_to_max_length=True, return_attention_mask=True,
                return_tensors='pt'
            )

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1938: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': tensor([[ 101, 1045, 2572, 1037, 2204, 2711, 1010, 1998, 2129, 2055, 2017, 1029,
          102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [9]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### XLNet

In [1]:
import torch

In [2]:
from transformers import XLNetTokenizer
from transformers import XLNetModel, XLNetForSequenceClassification

In [4]:
model = XLNetModel.from_pretrained('xlnet-base-cased', output_hidden_states=True)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
model

XLNetModel(
  (word_embedding): Embedding(32000, 768)
  (layer): ModuleList(
    (0): XLNetLayer(
      (rel_attn): XLNetRelativeAttention(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): XLNetFeedForward(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_1): Linear(in_features=768, out_features=3072, bias=True)
        (layer_2): Linear(in_features=3072, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): XLNetLayer(
      (rel_attn): XLNetRelativeAttention(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): XLNetFeedForward(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_1): Linear(in_features=768, out_features=3072, b

In [6]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

In [14]:
text = 'I am a good boy'

In [15]:
tokenizer.tokenize(text)

['▁', 'i', '▁am', '▁a', '▁good', '▁boy']

In [22]:
encoded_dict = tokenizer.encode_plus(
    "<cls> asd11l2js I am a good boy", add_special_tokens=True, max_length=50,
    pad_to_max_length=True, return_attention_mask=True,
    return_tensors='pt'
)

In [23]:
encoded_dict

{'input_ids': tensor([[   5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,
            5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,
            5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    3,   34,
           66, 1545,  368,  184, 1315,   23,   17,  150,  569,   24,  195, 2001,
            4,    3]]), 'token_type_ids': tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 2]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [24]:
text_list, attention_masks = list(), list()
text_list.append(encoded_dict['input_ids'])
attention_masks.append(encoded_dict['attention_mask'])
text_list = torch.cat(text_list, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

In [10]:
outputs = model(text_list, attention_masks)

In [11]:
outputs

XLNetModelOutput(last_hidden_state=tensor([[[ 1.4764,  2.5515, -0.8054,  ..., -0.7408,  0.5854, -2.2659],
         [ 1.6141,  2.4739, -0.7761,  ..., -0.7215,  0.4234, -2.3880],
         [ 1.5958,  2.4999, -0.7793,  ..., -0.6517,  0.2706, -2.4049],
         ...,
         [ 3.8463,  0.2613, -1.5299,  ..., -2.6709, -0.2188,  0.2802],
         [ 5.7440, -0.2734, -3.4873,  ..., -3.3266,  0.4171,  0.4994],
         [ 3.7563, -0.2951, -2.6874,  ..., -1.8446,  0.5208,  0.9403]]],
       grad_fn=<PermuteBackward>), mems=(tensor([[[ 0.0344,  0.0202,  0.0261,  ..., -0.0175, -0.0343,  0.0252]],

        [[ 0.0344,  0.0202,  0.0261,  ..., -0.0175, -0.0343,  0.0252]],

        [[ 0.0344,  0.0202,  0.0261,  ..., -0.0175, -0.0343,  0.0252]],

        ...,

        [[ 0.0540,  0.0375,  0.0332,  ...,  0.0201, -0.0480,  0.0724]],

        [[ 0.0788, -0.0583, -0.0905,  ...,  0.0493,  0.0634, -0.0520]],

        [[ 0.0181, -0.0015, -0.1494,  ...,  0.0012, -0.0009,  0.0188]]]), tensor([[[ 0.9156,  1.0634, -

In [13]:
outputs.hidden_states

(tensor([[[ 0.0344,  0.0202,  0.0261,  ..., -0.0175, -0.0343,  0.0252],
          [ 0.0344,  0.0202,  0.0261,  ..., -0.0175, -0.0343,  0.0252],
          [ 0.0344,  0.0202,  0.0261,  ..., -0.0175, -0.0343,  0.0252],
          ...,
          [ 0.0540,  0.0375,  0.0332,  ...,  0.0201, -0.0480,  0.0724],
          [ 0.0788, -0.0583, -0.0905,  ...,  0.0493,  0.0634, -0.0520],
          [ 0.0181, -0.0015, -0.1494,  ...,  0.0012, -0.0009,  0.0188]]],
        grad_fn=<PermuteBackward>),
 tensor([[[ 0.9156,  1.0634, -0.2741,  ..., -0.1662, -0.1363, -0.8428],
          [ 0.9461,  1.0958, -0.2464,  ..., -0.1756, -0.1703, -0.8440],
          [ 0.9758,  1.1076, -0.2352,  ..., -0.1836, -0.1847, -0.8192],
          ...,
          [ 0.8328,  0.5979,  0.6304,  ..., -0.4727, -1.2793,  1.6664],
          [ 1.0250, -0.4521, -1.5119,  ..., -0.0787,  0.4344,  0.7233],
          [ 0.1938,  0.1017, -2.3535,  ..., -0.5091, -0.1052,  1.6627]]],
        grad_fn=<PermuteBackward>),
 tensor([[[ 0.3933,  0.3043, -

In [ ]:
outputs

## Evaluation